In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
import pickle
import surprise as srp

In [ ]:
# path to dataset file
#file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
# and split it into 3 folds for cross-validation.
reader = srp.Reader(line_format='user item rating timestamp', sep=',')

data = srp.Dataset.load_from_file('data/training.dat', reader=reader)
data.split(n_folds=3)


In [ ]:
# We'll use the famous SVD algorithm.
algo = srp.SVD()

for trainset, testset in data.folds():

    # train and test algorithm.
    algo.train(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    rmse = srp.accuracy.rmse(predictions, verbose=True)

In [ ]:
uid = str(1)  # raw user id (as in the ratings file). They are **strings**!
iid = str(48)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

In [ ]:
predictions[1]

In [ ]:
pred = algo.predict(uid, iid, r_ui=5, verbose=True)

In [ ]:
pred = algo.predict(uid, iid, r_ui=1, verbose=True)

In [ ]:
pred = algo.predict(uid, iid, r_ui=0, verbose=True)

In [ ]:
print predictions[1]

In [ ]:
print predictions[1][0]

In [ ]:
test_file = pd.read_table('data/test.csv', sep = ',', header=None, engine='python')
print test_file.shape
movie_file = pd.read_table('ml-1m/movies.dat', sep = '::', header=None, engine='python')
print(movie_file.shape)

In [ ]:
#movies 3666(gercege karşılık gelen index) alıp 3952(gerçekid) döner, movie_indices 3952 alıp 3666 döner
test_users = np.unique(test_file[0]) # 1(0.idex) den 6040(6039.index) a kadar
movies = np.unique(movie_file[0])

test_number_of_rows = len(test_users) #6040
number_of_columns = len(movies) #3667

movie_indices, test_user_indices = {}, {}
 
for i in range(len(movies)):
    movie_indices[movies[i]] = i # movie_indices[3952] = 3666 x.filmin indisini verir
  
for i in range(len(test_users)):
    test_user_indices[test_users[i]] = i # x.userın indisini verir
print(len(movie_indices))

In [ ]:
V = sp.lil_matrix((test_number_of_rows, number_of_columns))
for line in test_file.values:
    u, i , r , time = map(int,line)
    V[test_user_indices[u], movie_indices[i]] = r # gerçek user ve movie idnin indexini bulup ratingi matrixteki yere atar

In [ ]:
P = sp.lil_matrix((test_number_of_rows, number_of_columns))
for user in range(test_number_of_rows):
    for movie in np.nonzero(V[user,:])[1]:
        pred = algo.predict(str(test_users[user]), str(movies[movie]), r_ui=0, verbose=False)
        P[user, movie] = pred[3]

In [ ]:
def recommend(index):
    indexList = np.nonzero(P[index,:])[1]
    relevant = P[index,indexList]
    print(indexList)
    print(relevant)
    indexSort = relevant.argsort()
    print(indexSort)
    for i in indexList:
        print(i)
recommend(0)

In [ ]:
x = np.argsort(P, axis=0)
print x